In [0]:

import psycopg2
from datetime import datetime, timedelta
from pprint import pprint
from aadatapipelinecore.core.utils.date_ import to_date, str_date
def gen_date_range(start_date, end_date):
    date_list = []
    start = to_date(start_date)
    end = to_date(end_date)
    d = start
    while d <= end:
        date_list.append(str_date(d))
        d = d + timedelta(days=1)
    return date_list
citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_store_db",
        user="citus_bdp_prod_int_datapipeline",
        host="10.2.10.254",
        password="PZQYyjRHCXZ23LVQ",
        port=5432
    )
)
def query(dsn, sql):
    # start_time = datetime.now()
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    # end_time = datetime.now()
    # cost = end_time - start_time
    return result
sql = """
SELECT
    granularity, date, device_code, country_code, publisher_id,
    sum(est_free_app_download) AS est_free_app_download
FROM (
    SELECT
        est.*, app.publisher_id, app.category_id
    FROM (
        SELECT *
        FROM store.store_est_fact_v6
        WHERE
            granularity = 'daily'
            AND date = '2020-05-16'
            AND device_code = 'ios-phone'
            AND country_code = 'US'
    ) AS est
    LEFT JOIN store.store_dna_map_dimension_v8 AS app
    ON est.app_id = app.app_id
) AS a
WHERE
    category_id = 100000
    AND est_free_app_download IS NOT NULL
GROUP BY granularity, date, device_code, country_code, publisher_id
ORDER BY est_free_app_download DESC
--LIMIT 1000
;
"""
# .format(date_range=tuple(gen_date_range("2020-05-15", "2020-05-16")))
# print sql
result = query(citus_dsn_, sql)
from pyspark.sql import types as T
from pyspark.sql import functions as F, window
columns = ["granularity", "date", "device_code", "country_code", "publisher_id", "est_free_app_download"]
db_publisher = spark.createDataFrame(result, columns)
db_publisher = (
    db_publisher
    .withColumn("est_free_app_download", db_publisher["est_free_app_download"].cast(T.IntegerType()))
    .withColumn("rank", F.row_number().over(window.Window.orderBy(F.col("est_free_app_download").desc())).cast(T.LongType()))
)
print db_publisher.count()
db_publisher.show(10, False)


In [0]:

df_publisher = (
    spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-05-16/")
    .where("platform = 'ios' AND feed = 0 AND store_id = 143441 AND category_id = 36")
    .select(["rank", "id", "est"])
    .withColumnRenamed("id", "publisher_id")
    .withColumnRenamed("est", "est_free_app_download")
    .orderBy("rank")
    # .limit(10000)
)
print df_publisher.count()
df_publisher.show(10, False)
Collapse



In [0]:

joined_df = (
    df_publisher
    .join(
        (
            db_publisher.select(["rank", "publisher_id", "est_free_app_download"])
            .withColumnRenamed("rank", "db_rank")
            .withColumnRenamed("est_free_app_download", "db_est_free_app_download")
        ),
        ["publisher_id"],
        "fullouter"
    )
    .orderBy("rank")
)
joined_df = (
    joined_df
    .withColumn("is_rank_same", F.lit(joined_df.rank == joined_df.db_rank))
    .withColumn("is_est_same", F.lit(joined_df.est_free_app_download == joined_df.db_est_free_app_download))
)
diff = joined_df.where("is_est_same == False OR is_est_same IS NULL").orderBy(["rank", "db_rank"])
print diff.count()
diff.show(100, False)
# 1. both in top, and rank is same
diff_1 = diff.where("is_rank_same == True").orderBy("rank")
print "1. both in top, and rank is same, {}".format(diff_1.count())
diff_1.show(100, False)
# 2. both in top, and rank is different
diff_2 = diff.where("is_rank_same == False").orderBy("rank")
print "2. both in top, and rank is different, {}".format(diff_2.count())
diff_2.show(100, False)
# 3. publisher in DF top, but not in DB top
diff_3 = diff.where("is_rank_same IS NULL and rank IS NOT NULL").orderBy("rank")
print "3. publisher in DF top, but not in DB top, {}".format(diff_3.count())
diff_3.show(100, False)
# 4. publisher not in DF top, but in DB top
diff_4 = diff.where("is_rank_same IS NULL and db_rank IS NOT NULL").orderBy("db_rank")
print "4. publisher not in DF top, but in DB top, {}".format(diff_4.count())
diff_4.show(100, False)
Collapse